In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import re
import string
import unicodedata
import plotly_express as px
import plotly.io as pio
from sklearn import preprocessing
import pandas_profiling
import datetime
%matplotlib inline
import matplotlib.style
import matplotlib as mpl
import matplotlib.pyplot as plt 
from sklearn import preprocessing
import warnings
import math

warnings.filterwarnings("ignore")


In [2]:
pwd

'/Users/joaocamargo/macHD/GITHUB/TCC-PUCRS-2020'

In [3]:
ls -ltr Data/

total 9424
-rw-r--r--  1 joaocamargo  staff  3301959 Apr 21 10:13 count_day_type.csv
-rw-r--r--  1 joaocamargo  staff   124920 Apr 21 10:13 count_day.csv
-rw-r--r--  1 joaocamargo  staff    24972 Apr 21 10:13 count_week_day_type.csv
-rw-r--r--  1 joaocamargo  staff      138 Apr 21 10:13 count_week_day.csv
-rw-r--r--  1 joaocamargo  staff   386439 Apr 21 10:13 count_year_month_type.csv
-rw-r--r--  1 joaocamargo  staff     4827 Apr 21 10:13 count_year_month.csv


In [4]:
df_count_day_type = pd.read_csv('./Data/count_day_type.csv',sep=",",encoding='ascii')
df_count_week_day_type = pd.read_csv('Data/count_week_day_type.csv',sep=",",encoding='ascii')
df_count_year_month_type = pd.read_csv('Data/count_year_month_type.csv',sep=",",encoding='ascii')

del df_count_day_type['Unnamed: 0']
#del df_count_day['Unnamed: 0']
del df_count_week_day_type['Unnamed: 0']
#del df_count_week_day['Unnamed: 0']
del df_count_year_month_type['Unnamed: 0']
#del df_count_year_month['Unnamed: 0']

In [5]:
#dataFrames = [df_count_day_type,df_count_day,df_count_week_day_type,df_count_week_day,df_count_year_month_type,df_count_year_month]
dataFrames = [df_count_day_type,df_count_week_day_type,df_count_year_month_type]
for df in dataFrames:
    display(df.head(1))

,judgmentDate,type,count
0,2000-01-03,Agravo de Instrumento,2


,weekDay,type,count
0,Sun,A\xc3\xa7\xc3\xa3o Civil P\xc3\xbablica,2


,yearMonth,type,count
0,2000-01-01,Agravo de Instrumento,12


In [6]:
#verificar feriados 
def isHoliday(dia,mes):
    if dia == 1:
        if mes == 1 or mes ==5:
            return(True)
    if (dia ==2 or dia==15) and mes==11:
        return(True)
    if (dia ==7 or dia==20) and mes == 9:
        return(True)
    if mes == 4 and (dia ==10 or dia ==12 or dia==21):
        return(True)
    if dia ==24 and mes == 6:
        return(True)
    if mes==10 and (dia==12 or dia==28):
        return(True)
    if mes==12 and dia ==25:
        return(True)
    return(False)   

In [7]:
#transformar data 
# adiciona coluna de dia da semana  (segunda=0,terça=1,quarta=2,quinta=3, sexta=4,sabado=5,domingo=6)
for df in [df_count_day_type]:
    if df['judgmentDate'].dtype != 'datetime64[ns]':
        character = df['judgmentDate'].str.replace('\d+','')[2][1]
        formato=''
        print('formato: ',df['judgmentDate'].str.find(character)[0])
        if df['judgmentDate'].str.find(character)[0] ==4:
            formato = '%Y{0}%m{0}%d'
        else:
            formato = '%d{0}%m{0}%Y'
        print(character)
    df['judgmentDate'] = pd.to_datetime(df['judgmentDate'],format=formato.format(character))
    df['weekDay'] = df['judgmentDate'].apply(lambda x: x.weekday())
    df['weekType'] = df['judgmentDate'].apply(lambda x: 'DiaSemana' if x.weekday()<5 else 'FimSemana')
    df['Dia'] = df['judgmentDate'].apply(lambda x: x.day)   
    df['Mes'] = df['judgmentDate'].apply(lambda x: x.month)  
    df['Ano'] = df['judgmentDate'].apply(lambda x: x.year)  
    df['Feriado'] = df['judgmentDate'].apply(lambda x: isHoliday(x.day,x.month))  

formato:  4
-


In [8]:
def elementdecode(x):
    #if the first condition if not met, the second is not evaluated
    if (type(x) == str):        
        bb=x.encode()
        cc = bb.decode('unicode-escape').encode('ISO-8859-1')                
        return(cc.decode('utf-8'))
    else:                
        return x.decode('utf-8')

In [9]:
df_count_day_type['type'] = df_count_day_type['type'].apply(lambda x: elementdecode(x))
df_count_week_day_type['type'] = df_count_week_day_type['type'].apply(lambda x: elementdecode(x))
df_count_year_month_type['type'] = df_count_year_month_type['type'].apply(lambda x: elementdecode(x))

In [10]:
for df in dataFrames:
    display(df.head(1))

,judgmentDate,type,count,weekDay,weekType,Dia,Mes,Ano,Feriado
0,2000-01-03,Agravo de Instrumento,2,0,DiaSemana,3,1,2000,False


,weekDay,type,count
0,Sun,Ação Civil Pública,2


,yearMonth,type,count
0,2000-01-01,Agravo de Instrumento,12



<div class="alert alert-info"><h1> Export dos CVS já tratados
</h1>
    <hr/>
</div>


In [11]:
df_count_day_type.to_csv('./data_joao/df_count_day_type_modificado.csv')
df_count_week_day_type.to_csv('./data_joao/df_count_week_day_type_modificado.csv')
df_count_year_month_type.to_csv('./data_joao/df_count_year_month_type_modificado.csv')